<a href="https://colab.research.google.com/github/Yash-Yelave/Recomendation_systems/blob/main/RS_PR10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
from collections import defaultdict

# ==============================================================================
# 1. MOCK DATA & METADATA
# ==============================================================================

# --- Item Metadata ---
item_metadata = {
    'item1': {'category': 'electronics', 'price': 100}, 'item2': {'category': 'apparel', 'price': 80},
    'item3': {'category': 'electronics', 'price': 120}, 'item4': {'category': 'books', 'price': 25},
    'item5': {'category': 'books', 'price': 30}, 'item6': {'category': 'books', 'price': 20},
    'item7': {'category': 'apparel', 'price': 75}, 'item8': {'category': 'books', 'price': 60},
    'item9': {'category': 'electronics', 'price': 45}, 'item10': {'category': 'books', 'price': 15}
}
all_items = list(item_metadata.keys())

# --- User Interaction History (TRAINING DATA) ---
# This is what the model "knows" about the user's past behavior.
user_interaction_history = {
    'user1': ['item1', 'item6'], # User 1 likes electronics and books
    'user2': ['item2', 'item7'], # User 2 likes apparel
    'user3': ['item4', 'item5'], # User 3 likes books
}

# --- Ground Truth (TEST DATA) ---
# These are the *unseen* items that the user actually likes.
# A good model should recommend these.
ground_truth_ratings = {
    'user1': {'item3': 5, 'item9': 4},
    'user2': {'item4': 4, 'item5': 5},
    'user3': {'item1': 3, 'item2': 4, 'item6': 5, 'item10': 4}
}
ground_truth_top_n = {
    'user1': ['item3', 'item9'],
    'user2': ['item4', 'item5'],
    'user3': ['item1', 'item6', 'item8', 'item10']
}

# ==============================================================================
# 2. PLACEHOLDER RECOMMENDATION SYSTEM ALGORITHMS
# ==============================================================================

def collaborative_filtering_model(user_id):
    """Simulates a collaborative filtering model that predicts ratings for unseen items."""
    # Note: A real CF model would be trained on a larger dataset.
    # This mock output is designed to have some accuracy for evaluation.
    predictions = {
        'user1': {'item3': 4.8, 'item9': 3.5, 'item2': 2.5, 'item4': 3.1},
        'user2': {'item4': 4.1, 'item5': 4.9, 'item1': 3.0},
        'user3': {'item1': 3.3, 'item2': 4.5, 'item6': 4.1, 'item10': 3.9, 'item8': 3.2}
    }
    return predictions.get(user_id, {})

def content_based_model(user_id):
    """(IMPROVED) Recommends items based on the user's interaction history."""
    history = user_interaction_history.get(user_id, [])
    if not history: return []

    liked_categories = {item_metadata[item]['category'] for item in history if item in item_metadata}

    scores = defaultdict(float)
    for item in all_items:
        if item in item_metadata and item_metadata[item]['category'] in liked_categories:
            scores[item] += 1

    # Filter out items the user has already seen
    recommendations = {item: score for item, score in scores.items() if item not in history}
    return sorted(recommendations.keys(), key=lambda x: recommendations[x], reverse=True)

def knowledge_based_model(user_requirements):
    """Filters based on explicit rules."""
    recs = []
    for item, metadata in item_metadata.items():
        if all([
            metadata.get('category') == user_requirements.get('category', metadata.get('category')),
            metadata.get('price', float('inf')) <= user_requirements.get('max_price', float('inf'))
        ]):
            recs.append(item)
    return recs

def hybrid_model(user_id):
    """Simulates a hybrid model combining CF scores and CBF ranking."""
    history = user_interaction_history.get(user_id, [])
    cf_preds = collaborative_filtering_model(user_id)
    cb_preds_list = content_based_model(user_id)
    cb_scores = {item: len(cb_preds_list) - i for i, item in enumerate(cb_preds_list)}

    hybrid_scores = defaultdict(float)
    max_cf = max(cf_preds.values()) if cf_preds else 1
    for item, score in cf_preds.items():
        hybrid_scores[item] += 0.5 * (score / max_cf)

    max_cb = max(cb_scores.values()) if cb_scores else 1
    for item, score in cb_scores.items():
        hybrid_scores[item] += 0.5 * (score / max_cb)

    filtered = {item: score for item, score in hybrid_scores.items() if item not in history}
    return sorted(filtered.keys(), key=lambda x: filtered[x], reverse=True)

# ==============================================================================
# 3. EVALUATION FUNCTIONS
# ==============================================================================

def evaluate_rating_prediction(y_true_ratings, y_pred_ratings_all_users):
    """Calculates MAE and RMSE."""
    true, pred = [], []
    for user, items in y_true_ratings.items():
        for item, rating in items.items():
            if user in y_pred_ratings_all_users and item in y_pred_ratings_all_users[user]:
                true.append(rating)
                pred.append(y_pred_ratings_all_users[user][item])
    if not true: return {"mae": float('nan'), "rmse": float('nan')}
    return {"mae": mean_absolute_error(true, pred), "rmse": np.sqrt(mean_squared_error(true, pred))}

def calculate_ranking_metrics(y_true_top_n, y_pred_top_n, k):
    """Calculates Accuracy, Precision, Recall, F1-Score, and NDCG at k."""
    metrics = defaultdict(list)
    for user, true_items in y_true_top_n.items():
        if user in y_pred_top_n and y_pred_top_n[user]:
            pred = y_pred_top_n[user][:k]
            true_set = set(true_items)
            hits = len(true_set.intersection(set(pred)))

            precision = hits / len(pred)
            recall = hits / len(true_set) if true_set else 0.0
            f1 = 2 * (precision * recall) / (precision + recall) if (precision + recall) > 0 else 0.0

            metrics['accuracy'].append(precision)
            metrics['precision'].append(precision)
            metrics['recall'].append(recall)
            metrics['f1_score'].append(f1)

            dcg = sum(1.0 / np.log2(i + 2) for i, item in enumerate(pred) if item in true_set)
            idcg = sum(1.0 / np.log2(i + 2) for i in range(min(k, len(true_set))))
            metrics['ndcg'].append((dcg / idcg) if idcg > 0 else 0.0)

    return {f"{name}@{k}": np.mean(values) for name, values in metrics.items()}

# ==============================================================================
# 4. MAIN EXECUTION & EVALUATION
# ==============================================================================

if __name__ == "__main__":
    K = 5
    print(f"🚀 Recommendation System Evaluation (k={K}) 🚀\n")

    # --- 4.1 Collaborative Filtering ---
    print("## 1. Collaborative Filtering Evaluation")
    cf_preds = {user: collaborative_filtering_model(user) for user in ground_truth_top_n}
    rating_metrics = evaluate_rating_prediction(ground_truth_ratings, cf_preds)
    print(f"  --> MAE:  {rating_metrics['mae']:.4f}, RMSE: {rating_metrics['rmse']:.4f} (Lower is better)")

    cf_top_n = {u: sorted(p.keys(), key=p.get, reverse=True) for u, p in cf_preds.items()}
    ranking_metrics = calculate_ranking_metrics(ground_truth_top_n, cf_top_n, K)
    for m, v in ranking_metrics.items(): print(f"  --> {m}: {v:.4f} (Higher is better)")
    print("-" * 45)

    # --- 4.2 Content-Based ---
    print("\n## 2. Content-Based Model Evaluation")
    print("  --> MAE/RMSE: Not applicable (does not predict ratings)")
    cb_top_n = {user: content_based_model(user) for user in ground_truth_top_n}
    ranking_metrics = calculate_ranking_metrics(ground_truth_top_n, cb_top_n, K)
    for m, v in ranking_metrics.items(): print(f"  --> {m}: {v:.4f} (Higher is better)")
    print("-" * 45)

    # --- 4.3 Knowledge-Based ---
    print("\n## 3. Knowledge-Based Model Evaluation")
    user3_reqs = {'category': 'books', 'max_price': 50}
    kb_top_n = {'user3': knowledge_based_model(user3_reqs)}
    print(f"  --> Evaluating for user3 with requirements: {user3_reqs}")
    print("  --> MAE/RMSE: Not applicable (does not predict ratings)")
    ranking_metrics = calculate_ranking_metrics(ground_truth_top_n, kb_top_n, K)
    for m, v in ranking_metrics.items(): print(f"  --> {m}: {v:.4f} (Higher is better)")
    print("-" * 45)

    # --- 4.4 Hybrid Model ---
    print("\n## 4. Hybrid Model Evaluation")
    print("  --> MAE/RMSE: Not applicable (does not predict ratings)")
    hybrid_top_n = {user: hybrid_model(user) for user in ground_truth_top_n}
    ranking_metrics = calculate_ranking_metrics(ground_truth_top_n, hybrid_top_n, K)
    for m, v in ranking_metrics.items(): print(f"  --> {m}: {v:.4f} (Higher is better)")
    print("\n✅ Evaluation complete.")

🚀 Recommendation System Evaluation (k=5) 🚀

## 1. Collaborative Filtering Evaluation
  --> MAE:  0.3375, RMSE: 0.4287 (Lower is better)
  --> accuracy@5: 0.6556 (Higher is better)
  --> precision@5: 0.6556 (Higher is better)
  --> recall@5: 1.0000 (Higher is better)
  --> f1_score@5: 0.7852 (Higher is better)
  --> ndcg@5: 0.9202 (Higher is better)
---------------------------------------------

## 2. Content-Based Model Evaluation
  --> MAE/RMSE: Not applicable (does not predict ratings)
  --> accuracy@5: 0.7000 (Higher is better)
  --> precision@5: 0.7000 (Higher is better)
  --> recall@5: 0.8750 (Higher is better)
  --> f1_score@5: 0.7143 (Higher is better)
  --> ndcg@5: 0.8411 (Higher is better)
---------------------------------------------

## 3. Knowledge-Based Model Evaluation
  --> Evaluating for user3 with requirements: {'category': 'books', 'max_price': 50}
  --> MAE/RMSE: Not applicable (does not predict ratings)
  --> accuracy@5: 0.5000 (Higher is better)
  --> precision@5: 